In [16]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [18]:
subj = subject.Subject & {'subject_nickname': 'CSHL_007'}
# get trial counts and session length to date
session_info = (behavior.BehavioralSummaryByDate * 
                behavior.BehavioralSummaryByDate.ReactionTimeByDate & 
                subj).proj('session_date', 'performance_easy', 'median_reaction_time').fetch(as_dict=True)
session_info = pd.DataFrame(session_info)

# get the first date when animal became "trained" and "ready for ephys"
status = putils.get_status(subj)

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:151: UserWarning:

Mysql server has gone away. Reconnecting to the server.



In [20]:
yrange = [0, 1.1]
performance_easy = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['performance_easy'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='black'),
    name='performance easy',
    yaxis='y1'
)
rt = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['median_reaction_time'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='red'),
    name='reaction time',
    yaxis='y2'
)

if status['is_trained']:
    first_trained_plot=go.Scatter(
        x=[status['first_trained_date'], status['first_trained_date']],
        y=yrange,
        mode="lines",
        marker=dict(
            color='orange'),
        name='first day got trained'
    )

if status['is_biased']:
    first_biased_plot=go.Scatter(
        x=[status['first_biased_date'], status['first_biased_date']],
        y=yrange,
        mode="lines",
        marker=dict(
            color='green'),
        name='first day got biased'
    )

data = [performance_easy, rt]

if status['is_trained']:
    data.append(first_trained_plot)

if status['is_biased']:
    data.append(first_biased_plot)

layout = go.Layout(
    yaxis=dict(
        title='Performance on easy trials',
        range=yrange
    ),
    yaxis2=dict(
        title='Median reaction time (s)',
        overlaying='y',
        color = 'red',
        side='right',
        type='log',
        range=np.log10([0.1, 10]).tolist(),
        dtick=np.log10([0.1, 1, 10]).tolist()
        
    ),
    xaxis=dict(
        title='Date',
    ), 
    width=500,
    height=400,
    title='Performance and median reaction time',
    legend=dict(
        x=0, 
        y=1.2,
        orientation='h')
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [15]:
f = open("performance_reaction_time.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()